In [41]:
from __future__ import division 

import gym
import numpy as np
import random
import tensorflow as tf 
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as initializer
import matplotlib.pyplot as plt
import scipy.misc
import os
import sys
%matplotlib inline

In [42]:
from gridworld_v04 import gameEnv
#env = gameEnv(partial=False, size=15)
#env.renderEnv()

In [43]:
class Agent():
    def __init__(self, hiddens = [256, 256, 256], num_frames = 4, state_size = 10,
                 action_size = 6, env=None, lr = 1e-3, batch_size = 32, num_episodes = 5000, pre_train_steps = 10000,
                 max_ep_length = 300, buffer_size = 50000, start_e = 1, final_e = 0.01, gamma = 0.99,
                 tau=0.001, update_freq = 5, load_model = False , logs_dir = "./logs_dir"):

        self.hiddens = hiddens # Size of the hidden layers
        self.num_frames = num_frames # Number of consecutive state frames
        self.state_size = state_size # Size of the state vector
        self.action_size = action_size  # Number of actions
        self.env = env # Save the environmet!
        self.lr=lr # learning rate of the optimizer
        self.batch_size=batch_size # Size of the experience sample batch
        self.num_episodes = num_episodes # Number of game environmet episodes in which we train
        self.pre_train_steps = pre_train_steps
        self.max_ep_length = max_ep_length # Maximun length (number of actions) of a single train episode
        self.buffer_size = buffer_size # Size of the experience replay buffer
        self.start_e = start_e # Inital value of the exploration coefficient
        self.final_e = final_e # Final value of the exploration coefficient
        self.gamma = gamma # Discount factor on the target Q-value
        self.tau = tau # Porcentage that determines how much are parameters of mainQN net modified by targetQN
        self.update_freq = update_freq # Frecuency of updates of the double DQN
        self.logs_dir = logs_dir # Path to store logs and checkpoints

        tf.reset_default_graph()

        # Instantiate the networks
        #self.mainQN = DQN("mainQN", state_size, action_size, hiddens, num_frames)
        self.mainQN = DQN("mainQN", len(self.env.objects) * 2, self.env.actions, hiddens, num_frames)
        self.targetQN = DQN("targetQN", len(self.env.objects) * 2, self.env.actions, hiddens, num_frames)

        init = tf.global_variables_initializer()
        trainables = tf.trainable_variables()

        self.target_ops = self.set_target_graph_vars(trainables, tau)

        # Create a experience replay buffer & score records
        self.exp_buffer = ExperienceBuffer()
        self.step_record = []
        self.reward_record = []

        self.model_saver = ModelSaver(self.logs_dir)
        
    def learn(self):
        print("Model Learning")
        init = tf.global_variables_initializer()
        sess = tf.Session()
        e = self.start_e
        current_step = 0 # previously total_steps

        self.model_saver = ModelSaver(self.logs_dir)

        with sess:

            sess.run(init)
           # if self.load_model == True:
           #     print('Loading Model...')
           #     model_saver.restore_model(sess)
           #     #ckpt = model_saver_restore_model(sess)

            # Set the target network to be equal to the primary network
            self.update_target_graph(self.target_ops, sess)

            # Start the pre train proces
            for episode in range(self.num_episodes):

                if episode % 100 == 0:
                    print("\n=====" + "Episode " + str(episode) + "starts =====" )

                episode_exp = ExperienceBuffer()

                #Reset environment and get first new observation
                s = self.env.reset()

                d = False # episode's "done" signal
                episode_reward_sum = 0
                episode_steps = 0

                #The Q-Network
                while episode_steps < self.max_ep_length: #If the agent take too long to win, end the trial.
                    episode_steps += 1

                    #Choose an action by greedily (with e chance of random action) from the Q-network
                    if np.random.rand(1) < e or current_step < self.pre_train_steps:
                        a = np.random.randint(0,4)
                    else:
                        a = sess.run(self.mainQN.predict,feed_dict={self.mainQN.state:[s]})[0]

                    s1,r,d = self.env.step(a)
                    current_step += 1
                    episode_exp.add(np.reshape(np.array([s,a,r,s1,d]),[1,5])) # Save the experience to our episode buffer.

                    # Start train process
                    if current_step > self.pre_train_steps:

                        if e > self.final_e:
                            stepDrop = 1/10000
                            e -= stepDrop

                        if current_step % self.update_freq == 0:

                            train_batch = self.exp_buffer.sample(self.batch_size) #Get a random batch of experiences.

                            #Perform the Double-DQN update to the target Q-values
                            Q1 = sess.run(self.mainQN.predict,
                                          feed_dict={self.mainQN.state:np.vstack(train_batch[:,3])})

                            Q2 = sess.run(self.targetQN.Qout,
                                          feed_dict={self.targetQN.state:np.vstack(train_batch[:,3])})

                            end_multiplier = -(train_batch[:,4] - 1)
                            doubleQ = Q2[range(self.batch_size),Q1]
                            targetQ = train_batch[:,2] + (self.gamma*doubleQ*end_multiplier)

                            # Update the network with our target values.
                            _ = sess.run(self.mainQN.updateModel,
                                         feed_dict={self.mainQN.state:np.vstack(train_batch[:,0]),
                                         self.mainQN.targetQ:targetQ,
                                         self.mainQN.actions:train_batch[:,1]})

                            # Set the target network to be equal to the primary
                            self.update_target_graph(self.target_ops, sess)

                    episode_reward_sum += r
                    s = s1

                    if d == True:
                        break

                self.exp_buffer.add(episode_exp.buffer)
                self.step_record.append(episode_steps)
                self.reward_record.append(episode_reward_sum)

                #Periodically save the model.
                if episode % 1000 == 0:
                    self.model_saver.save_model(sess, episode)
                    print("Model save_model")

                if len(self.reward_record) % 10 == 0:
                    print(current_step, np.mean(self.reward_record[-10:]), e)
            ##
            self.model_saver.save_model(sess, self.num_episodes)
        print("Percent of succesful episodes: " + str(100*sum(self.reward_record)/self.num_episodes) + "%")

    """ Auxiliary Methods """
    # Originally called updateTargetGraph
    def set_target_graph_vars(self, tfVars, tau):
        total_vars = len(tfVars)
        op_holder = []

        for idx,var in enumerate(tfVars[0:total_vars//2]): # Select the first half of the variables (mainQ net)
            op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau)+((1-tau)*tfVars[idx+total_vars//2].value())))

        return op_holder

    
    # Originally called updateTarget
    def update_target_graph(self, op_holder, sess):
        for op in op_holder:
            sess.run(op)
         
    def test(self):
        step_record_t = []
        reward_record_t = []
        current_steps_t = 0
        
        load_model = True
        num_test_episodes = 3000
        
        
        with tf.Session() as sess:
            if load_model == True:
                print('Loading Model...')
                self.model_saver.restore_model(sess)
                
            for episode in range(num_test_episodes):
                s = self.env.reset_for_testing()
                d = False
                episode_steps_t = 0
                episode_reword_sum_t = 0

                #Q-Network
                while episode_steps_t < self.max_ep_length: 
                    episode_steps_t+=1
                    a = sess.run(self.mainQN.predict, feed_dict={self.mainQN.state:[s]})[0]
                    s1, r, d = self.env.step_for_testing(a)
                    
                    current_steps_t += 1
                    episode_reword_sum_t += r
                    s = s1
                    
                    if d == True:
                        break
                        
                step_record_t.append(episode_steps_t)
                reward_record_t.append(episode_reword_sum_t)

                # print(episode_reword_sum_t)

                if len(reward_record_t) % 10 == 0:
                    accumR = np.mean(reward_record_t[-10:])
                    log = str(current_steps_t) +"\t"+ str(accumR) 
                    print(log)
                    #f.write(log)
                
    
        final_log = "Percent of sucessful episodes: " + str(sum(reward_record_t)/num_test_episodes) + "%"
        print(final_log)
        return reward_record_t
        


In [44]:
class DQN():
    def __init__(self, net_name, state_size, action_size, hiddens, num_frames):
        self.net_name = net_name

        with tf.variable_scope(self.net_name):

            #self.state = tf.placeholder(shape=[None, num_frames, state_size], dtype=tf.float32)
            self.state = tf.placeholder(shape=[None, state_size], dtype=tf.float32)
            #self.input_state = tf.reshape(self.state, [-1, num_frames * state_size])

            # Weights of each layer
            self.W = {
                'W1': self.init_weight("W1", [state_size, hiddens[0]]),
                'W2': self.init_weight("W2", [hiddens[0], hiddens[1]]),
                'W3': self.init_weight("W3", [hiddens[1], hiddens[2]]),
                'AW': self.init_weight("AW", [hiddens[2]//2, action_size]),
               #'AW': self.init_weight("AW", [hiddens[2]//2, hiddens[2]]),
                'VM': self.init_weight("VM", [hiddens[2]//2, 1])
            }

            self.b = {
                'b1': self.init_bias("b1", hiddens[0]),
                'b2': self.init_bias("b2", hiddens[1]),
                'b3': self.init_bias("b3", hiddens[2])
            }

            # Layers
            self.hidden1 = tf.nn.relu(tf.add(tf.matmul(self.state, self.W['W1']), self.b['b1']))
            self.hidden2 = tf.nn.relu(tf.add(tf.matmul(self.hidden1, self.W['W2']), self.b['b2']))
            self.hidden3 = tf.nn.relu(tf.add(tf.matmul(self.hidden2, self.W['W3']), self.b['b3']))

            # Compute the Advantage, Value, and total Q value
            self.A, self.V = tf.split(self.hidden3, 2, 1)
            self.Advantage = tf.matmul(self.A, self.W['AW'])
            self.Value = tf.matmul(self.V, self.W['VM'])
            self.Qout = self.Value + tf.subtract(self.Advantage, tf.reduce_mean(self.Advantage, axis=1, keep_dims=True))

            # Calcultate the action with highest Q value
            self.predict = tf.argmax(self.Qout, 1)

            # Compute the loss (sum of squared differences)
            self.targetQ = tf.placeholder(shape=[None], dtype=tf.float32)
            self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
            self.actions_one_hot = tf.one_hot(self.actions, action_size, dtype=tf.float32)

            self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_one_hot), axis=1)
            self.td_error = tf.square(self.targetQ - self.Q)
            self.loss = tf.reduce_mean(self.td_error)

            self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
            self.updateModel = self.trainer.minimize(self.loss)

    def init_weight(self, name, shape):
        return tf.get_variable(name=name, shape=shape, initializer=initializer.xavier_initializer())
                               #initializer=tf.contrib.layers.xavier_initializer())

    def init_bias(self, name, shape):
        return tf.Variable(tf.random_normal([shape]))
        #return tf.get_variable(name=name, shape=shape, initializer=tf.constant(np.random.rand(range(shape))))
                              # initializer= tf.random_initializer())

In [45]:
class ExperienceBuffer():
    def __init__(self, buffer_size = 50000):
        self.buffer = []
        self.buffer_size = buffer_size

    def add(self, experience):

        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience) + len(self.buffer)) - self.buffer_size] = []

        self.buffer.extend(experience)

    def sample(self, size):
        return np.reshape(np.array(random.sample(self.buffer, size)),[size,5])

In [46]:
class ModelSaver():
    def __init__(self, path):
        self.saver = tf.train.Saver()
        self.ckptPath = path
        if not os.path.exists(path):
            os.makedirs(path)

    def restore_model(self, sess):
        ckpt = tf.train.get_checkpoint_state(self.ckptPath)
        self.saver.restore(sess, ckpt.model_checkpoint_path)

    def save_model(self, sess, num_episode):
        self.saver.save(sess, self.ckptPath+'/model-'+str(num_episode)+'.cptk')

In [41]:
class LogManager():
    def __init__(self):
        return

    # If there is no file, create file with name firstly
    # If there is file with name, append content in the file 
    def write(self, name="default_filename", content=""):
        name = name+".txt"
        with open(name, 'a') as f:
            f.write(content)
        return
    def read(self, name="default_filename"):
        name = name+".txt"
        try:
            f = open(name, 'r')
            data = f.read()
            return data
        except FileNotFoundError as e:
            print(e)


In [47]:
class DrawGraph():
    def __init__(self, reward_record):
        self.reward_record = reward_record
  
    def plot(self):
        rMat = np.resize(np.array(self.reward_record), [len(self.reward_record)//100,100])
        rMean = np.average(rMat, 1)
        plt.plot(rMean)

In [45]:
def main():
    #env = gameEnv(partial = False, size=15)

    #agent = Agent([256, 256, 256], 4, 10, 6, env, 1e-3, 32, 10000, 100, 50000,
    #              1, 0.01, 10000, 0.99, 0.001, 5, False, "./log_dir")

    # The parameters of Agent class are as follows:
    # hiddens, num_frames, state_size, action_size, env, lr, batch_size, num_episodes, pre_train_steps,
    # max_ep_length, buffer_size, start_e, final_e, gamma, tau, update_freq, load_model, logs_dir
    
    name="test"
    log = LogManager()
    log.write(name, "13432432")
    print(log.read(name))
    
    """
    agent = Agent([256, 256, 256], 4, 10, 6, env, 1e-3, 32, 10000, 10000, 500, 50000, 1, 0.01, 0.99, 0.001, 5, False, "./logs_dir")
    agent.learn()
    
    reward_record_t = agent.test()
    graph = DrawGraph(reward_record_t)
    graph.plot()
    """
    
if __name__ == '__main__':
    main()


qweqweqwewqeqwewqe11113432432
